In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
from matplotlib import animation

from scipy.optimize import minimize, OptimizeResult
from collections import defaultdict
from itertools import zip_longest
from functools import partial

from answer import Answer

## <span style="color:blue">Implementation (Students do)</span>
<hr/>

### Methods
You will implement five optimization algorithms (descriptions available [here](https://ruder.io/optimizing-gradient-descent/index.html)). 
- Gradient descent (`gd`)
- Momentum gradient method (`momentum`)
- Nesterov's accelerated gradient method (`nag`)
- Adaptive gradient method (`adagrad`)
- Adaptive moment estimation (`adam`)

The last is a very common optimizer used in practical applications -- possibly the most common in the world.

Make note of the function headers: `def gd(func, x, lr, num_iters, jac, tol, callback, *args, **kwargs):`. Each method will satisfy this header format in accordance with the specification of custom minimizers used with `scipy.optimize.minimize`. This function is [well-documented](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html), but the highlights of the arguments are below.
- `func`: [type: function] The loss function. Takes in a point of type np.ndarray (2,) and returns a float representing the value of the function at that point.
- `x`: [type: np.ndarray (2,)] The starting point of the optimization.
- `lr`: [type: function] Learning rate schedule. Takes in an argument of type int representing the iteration number, and returns the learning rate to be used for that iteration.
- `num_iters`: [type: int] The number of iterations of the optimization method to run.
- `jac`: [type: function] The gradient of the loss function. "Jac" stands for Jacobian, which is out of scope for this class, but for scalar-valued functions, it is the transpose of the gradient. Takes in a point of type np.ndarray (2,) and returns an np.ndarray (2,) representing the gradient of the function at that point.
- `tol`: [type: float] The tolerance within which the optimization method is deemed to have converged.
- `callback`: [type: function] A function to be called on each iterate over the course of the optimization.
- `*args` and `**kwargs`: You will not need to use these, but they are present for compatibility with the `scipy.optimize.minimize` API.

Each function will need to return a two-tuple containing
- An instance of `scipy.optimize.OptimizeResult`, described [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.OptimizeResult.html).
- A `np.ndarray` containing the function value at the initial point and each iterate over the course of the optimization.

In [3]:
# demonstration of scipy.optimize.minimize
# minimize(lambda x: (x ** 2, 2 * x), 3, jac=True, method='Newton-CG', callback=print)
OptimizeResult(x=np.array([0,0]))

 x: array([0, 0])

In [16]:
np.ndarray(3)

array([0. , 0.5, 1. ])

In [28]:
def gd(func, x, lr, num_iters, jac, tol, callback, *args, **kwargs):
    # What is callback for?
    func_values = [func(x)]
    x_values = [x]
    for i in np.arange(num_iters):
        x_next = x_values[i] - lr(i) * jac(x_values[i])
        func_values.append(func(x_next))
        x_values.append(x_next)
        # x_values has added index
        print(np.array(x_values[i+1]))
        if np.linalg.norm(x_values[i+1] - x_values[i]) < tol:
            opt_result = OptimizeResult(x=x_values[-1], success=True, message="Converged", nit=1+i)
            return (opt_result, np.array(func_values))
    
    opt_result = OptimizeResult(x=x_values[-1], success=False, message="Failed to converge", nit=num_iters)
    return (opt_result, np.array(func_values))
    
    

def momentum(func, x, lr, num_iters, jac, tol, callback, gamma=0.9, *args, **kwargs):
    v_values = [0]
    func_values = [func(x)]
    x_values = [x]
    for i in np.arange(num_iters):
        v_next = gamma * v_values[i] + lr(i) * jac(x_values[i])
        x_next = x_values[i] - v_next
        func_values.append(func(x_next))
        x_values.append(x_next)
        v_values.append(v_next)
        # x_values has added index
        print(np.array(x_values[i+1]))
        if np.linalg.norm(x_values[i+1] - x_values[i]) < tol:
            opt_result = OptimizeResult(x=x_values[-1], success=True, message="Converged", nit=1+i)
            return (opt_result, np.array(func_values))
    
    opt_result = OptimizeResult(x=x_values[-1], success=False, message="Failed to converge", nit=num_iters)
    return (opt_result, np.array(func_values))
    

def nag(func, x, lr, num_iters, jac, tol, callback, gamma=0.9, *args, **kwargs):
    v_values = [0]
    func_values = [func(x)]
    x_values = [x]
    for i in np.arange(num_iters):
        v_next = gamma * v_values[i] + lr(i) * jac(x_values[i] - gamma * v_values[i])
        x_next = x_values[i] - v_next
        func_values.append(func(x_next))
        x_values.append(x_next)
        v_values.append(v_next)
        # x_values has added index
        print(np.array(x_values[i+1]))
        if np.linalg.norm(x_values[i+1] - x_values[i]) < tol:
            opt_result = OptimizeResult(x=x_values[-1], success=True, message="Converged", nit=1+i)
            return (opt_result, np.array(func_values))
    
    opt_result = OptimizeResult(x=x_values[-1], success=False, message="Failed to converge", nit=num_iters)
    return (opt_result, np.array(func_values))
    

def adagrad(func, x, lr, num_iters, jac, tol, callback, eps=1e-5, *args, **kwargs):
    
    pass

def adam(func, x, lr, num_iters, jac, tol, callback, beta1=0.9, beta2=0.999, eps=1e-5, *args, **kwargs):
    pass
    

### Functions and gradients
You have been given the implementation of four functions ($\mathbb{R}^2\to\mathbb{R}$), given below. You will need to implement `grad`, which returns their gradients as `np.ndarray` (2,). There is a field below for you to submit the gradient in $\LaTeX$.
- Booth function: $f_1(x)=\left(x_1+2x_2-7\right)^2+\left(2x_1+x_2-5\right)^2$
- Beale function: $f_2(x)=\left(1.5-x_1+x_1 x_2\right)^2+\left(2.25-x_1+x_1x_2^2\right)^2+\left(2.625-x_1+x_1 x_2^3\right)^2$
- Rosenbrock function: $f_3(x)=100\cdot\left(x_2-x_1^2\right)^2+\left(x_1-1\right)^2$
- Ackley function: $f_4(x)=-20\cdot\exp\left(-\frac{1}{5}\sqrt{\frac{x_1^2+x_2^2}{2}}\right)-\exp\left(\frac{\cos 2\pi x_1 + \cos 2\pi x_2}{2}\right)+20+\exp(1)$

In [6]:
def func(fn, x, y):
    if fn == 'booth':
        return (x + 2*y - 7)**2 + (2*x + y -5)**2
    elif fn == 'beale':
        return 0
    elif fn == 'rosen2d':
        return 0
    elif fn == 'ackley2d':
        return 0
    else:
        raise ValueError('Function %s not supported.' % fn)
        
def grad(fn, x, y):
    gradient = np.zeros_like(x)
    if fn == 'booth':
        g1 = 10*x + 8*y - 34
        g2 = 8*x + 10*y - 38
        np.array([g1, g2])
    elif fn == 'beale':
        g1 = 0
        g2 = 0
    elif fn == 'rosen2d':
        g1 = 0
        g2 = 0
    elif fn == 'ackley2d':
        g1 = 0
        g2 = 0
    else:
        raise ValueError('Function %s not supported.' % fn)
    return np.stack((g1, g2), axis=-1)

#### <span style="color:red">Submission: Gradient values $\LaTeX$</span>
<div style="color: red">
Enter the gradients you calculated below.
<ul>
<li>Booth: $\nabla_x f_1(x)=\text{replace me}$</li>
<li>Beale: $\nabla_x f_2(x)=\text{replace me}$</li>
<li>Rosenbrock: $\nabla_x f_3(x)=\text{replace me}$</li>
<li>Ackley: $\nabla_x f_4(x)=\text{replace me}$</li>
</ul>
</div>

## <span style="color:blue">Student-facing `Answer` class (provided)</span>

You have been provided a class called `Answer` which will be helpful for the remainder of the project. It can be found in the `answer.py` file. You are welcome to read and modify it, but this is not required. All information you need about this class is documented here, and examples of usage are given below.

### Documentation
- `__init__(self, methods, func, grad)`
  - Instantiates the `Answer` class with the functions you have implemented. `methods` is a dictionary mapping algorithm names to the functions that implement them, and `func` and `grad` are the functions of the same name that you have implemented.
- `set_fn_settings(self, fn_name)`
  - Sets the instance variables needed for visualizing `fn_name` with `plot2d` and `plot3d`. Needs to be called before calling these functions.
- `set_settings(self, fn_name, method, x0, **kwargs)`
  - Sets the instance variables needed for visualizing `method` optimizing `fn_name` starting at `x0` with `path2d`, `path3d`, `video2d`, and `video3d`. Any additional `kwargs` (likely `lr` and `num_iters`) will be passed on to `method`. Needs to be called before calling these functions or `compare`.
- `get_settings(self)`
  - Returns the arguments passed into `set_settings`: `fn_name`, `method`, `x0`, and `kwargs`.
- `compare(self, method, start_iter=0, **kwargs)`
  - Generates training loss graph comparing `method` with the previously set method on the previously set loss function and starting point, starting at iteration `start_iter`. Additional `kwargs` (likely `lr` and `num_iters`) will be passed on to `method`.
- `get_xs_losses(self)`
  - Returns a tuple containing
    - [type: `np.ndarray` (1 + `n_iters`, 2)] All iterates (including the initial point).
    - [type: `np.ndarray` (1 + `n_iters`,)] The loss at each iterate.
- `get_min_errs(self)`
  - Returns a tuple containing
    - `float` representing the closest (in L2 norm) the optimization procedure got to the global minimizer.
    - `float` representing the closest the optimization procedure got to the global minimum function value.
- `func_val(self, x)`
  - Returns `float` value of the previously set loss function evaluated at `x`. Convenience tool for debugging.
- `grad_val(self, x)`
  - Returns `np.ndarray` (2,) gradient of the previously set loss function evaluated at `x`. Convenience tool for debugging.
- `plot2d(self)`
  - Plots contours of the previously set loss function.
- `plot3d(self)`
  - Plots the previously set loss function.
- `path2d(self)`
  - Plots the sequence of iterates produced by the set method on the set loss function on a 2D contour.
- `path3d(self)`
  - Plots the sequence of iterates produced by the set method on the set loss function on a 3D graph. **NOTE:** This one does not work very well.
- `video2d(self, filename=None)`
  - Creates and saves an MP4 video of the path taken in `path2d` at `filename`. File name defaults to "{function}_{method}_2d.mp4"
- `video3d(self, filename=None)`
  - Creates and saves an MP4 video of the path taken in `path3d` at `filename`. File name defaults to "{function}_{method}_3d.mp4". **NOTE:** This works better than `path3d`.

In [ ]:
# instantiate the Answer class with the methods you have implemented! (You can implement and add more if you like!)
ans = Answer(
    {  # a mapping of algorithm names to functions implementing them
        'gd': gd,
        'momentum': momentum,
        'nag': nag,
        'adagrad': adagrad,
        'adam': adam
    },
    func,
    grad
)

## Testing your code
We are not providing much structure here, but now is a good time to make sure your optimization methods are working well. The cell below tests your gradient descent method on the function $f(x)=x^2$. We have included the output of our solution as a comment. Note that the function you feed it needs to take in a point as its sole argument and return the function as well as the gradient evaluated at that point.

In [24]:
# Maybe a useful starting example for testing gradient descent on a simple function
x_squared = lambda x: (x**2, 2*x)  # returns both the function value and the gradient.
opt_res, losses = minimize(x_squared, 3, jac=True, method=gd, callback=print,
                           options=dict(lr=lambda t: 0.25, x0=3, num_iters=15, tol=1e-3))
print('Final iterate: %.6f. Number of iterations: %d. Final loss: %.8f.' % (opt_res.x, opt_res.nit, losses[-1]))
# ----------------
# Expected output (GD):
# ----------------
# [1.5]
# [0.75]
# [0.375]
# [0.1875]
# [0.09375]
# [0.046875]
# [0.0234375]
# [0.01171875]
# [0.00585938]
# [0.00292969]
# [0.00146484]
# [0.00073242]
# Final iterate: 0.000732. Number of iterations: 12. Final loss: 0.00000054.

1.5
0.75
0.375
0.1875
0.09375
0.046875
0.0234375
0.01171875
0.005859375
0.0029296875
0.00146484375
0.000732421875
Final iterate: 0.000732. Number of iterations: 12. Final loss: 0.00000054.


In [29]:
x_squared = lambda x: (x**2, 2*x)  # returns both the function value and the gradient.
opt_res, losses = minimize(x_squared, 3, jac=True, method=momentum, callback=print,
                           options=dict(lr=lambda t: 0.12, x0=3, num_iters=15, tol=1e-10))
print('Final iterate: %.6f. Number of iterations: %d. Final loss: %.8f.' % (opt_res.x, opt_res.nit, losses[-1]))
# ----------------
# Expected output (Momentum):
# ----------------
# [2.28]
# [1.0848]
# [-0.251232]
# [-1.39336512]
# [-2.0868773]
# [-2.21018771]
# [-1.79072203]
# [-0.98342963]
# [-0.02084336]
# [0.85048669]
# [1.43056693]
# [1.60930308]
# [1.38393288]
# [0.8489558]
# [0.16372704]
# Final iterate: 0.163727. Number of iterations: 15. Final loss: 0.02680655.

2.2800000000000002
1.0848000000000002
-0.2512319999999997
-1.3933651199999997
-2.0868772991999998
-2.2101877086719997
-1.79072202711552
-0.9834296272069634
-0.02084335675959126
0.8504866922653456
1.4305669302441057
1.6093030811664046
1.3839328775165365
0.8489558036276865
0.16372704425707685
Final iterate: 0.163727. Number of iterations: 15. Final loss: 0.02680655.


In [30]:
x_squared = lambda x: (x**2, 2*x)  # returns both the function value and the gradient.
opt_res, losses = minimize(x_squared, 3, jac=True, method=nag, callback=print,
                           options=dict(lr=lambda t: 0.4, x0=3, num_iters=15, tol=1e-3))
print('Final iterate: %.6f. Number of iterations: %d. Final loss: %.8f.' % (opt_res.x, opt_res.nit, losses[-1]))
# ----------------
# Expected output (NAG):
# ----------------
# [0.6]
# [-0.312]
# [-0.22656]
# [-0.0299328]
# [0.02940634]
# [0.01656231]
# [0.00100054]
# [-0.00260101]
# [-0.00116848]
# [2.41592194e-05]
# [0.00021951]
# Final iterate: 0.000220. Number of iterations: 11. Final loss: 0.00000005.

0.5999999999999996
-0.31200000000000006
-0.22655999999999987
-0.029932799999999926
0.029406336000000005
0.016562311679999983
0.0010005379583999923
-0.0026010116782079987
-0.0011684812702310375
2.41592193896455e-05
0.00021950713200965199
Final iterate: 0.000220. Number of iterations: 11. Final loss: 0.00000005.


In [ ]:
x_squared = lambda x: (x**2, 2*x)  # returns both the function value and the gradient.
opt_res, losses = minimize(x_squared, 3, jac=True, method=adagrad, callback=print,
                           options=dict(lr=lambda t: 2, x0=3, num_iters=15, tol=1e-3))
print('Final iterate: %.6f. Number of iterations: %d. Final loss: %.8f.' % (opt_res.x, opt_res.nit, losses[-1]))
# ----------------
# Expected output (Adagrad):
# ----------------
# [1.00000028]
# [0.36754467]
# [0.13664342]
# [0.05087939]
# [0.01894909]
# [0.00705745]
# [0.00262851]
# [0.00097897]
# [0.00036461]
# Final iterate: 0.000365. Number of iterations: 9. Final loss: 0.00000013.

In [ ]:
x_squared = lambda x: (x**2, 2*x)  # returns both the function value and the gradient.
opt_res, losses = minimize(x_squared, 3, jac=True, method=adam, callback=print,
                           options=dict(lr=lambda t: 2, x0=3, num_iters=15, tol=1e-3))
print('Final iterate: %.6f. Number of iterations: %d. Final loss: %.8f.' % (opt_res.x, opt_res.nit, losses[-1]))
# ----------------
# Expected output (Adam):
# ----------------
# [1.00000333]
# [-0.74212166]
# [-1.76094027]
# [-1.93985392]
# [-1.565932]
# [-0.89684359]
# [-0.12388967]
# [0.58433522]
# [1.08297399]
# [1.292603]
# [1.21773996]
# [0.91871601]
# [0.47982495]
# [-0.00561154]
# [-0.44385421]
# Final iterate: -0.443854. Number of iterations: 15. Final loss: 0.19700656.

## Playground and Exploration

You are free to use the functions described above to explore the behavior of the optimization algorithms you have implemented. Pick different starting points, learning rate schedules, and even tolerances to explore! Example usage of the `Answer` class is below.

### Exploration
For each of the functions, start at the given initial points ($x_0$) and use any choice of optimization algorithm and associated hyperparameters to get within the specified distance of the global minimizer and minimum ($\epsilon_x$, $\epsilon_f$). *Hint: The `get_min_errs` function will be helpful*. There is a spot below for you to submit your results for each challenge.

- Booth function
  - $x_0=[8, 9], \epsilon_x=10^{-7}, \epsilon_f=10^{-14}$
- Beale function
  - $x_0=[3, 4], \epsilon_x=0.5, \epsilon_f=0.07$
- Rosenbrock function
  - $x_0=[8, 9], \epsilon_x=10^{-7}, \epsilon_f=10^{-14}$
- Ackley function
  - $x_0=[25, 20], \epsilon_x=2\cdot10^{-4}, \epsilon_f=5\cdot10^{-4}$. This function is hard. Tell us what you tried and how far you got.

In [ ]:
def lr_(t):
    if t < 5:
        return 1e-4
    elif t < 10:
        return 1e-2
    else:
        return 0.1
ans.set_settings(fn_name='booth', method='adam', x0=np.array([3, 4]), lr=lr_, num_iters=30, tol=1e-8)

In [ ]:
# How close the optimization got to the red star, and how far the min loss was from the global min.
print(ans.get_min_errs())

# The gradient at the end of the optimization. This can be helpful for tuning your learning rate schedule!
last_grad = ans.grad_val(ans.get_xs_losses()[0][-1])
print(last_grad)

# Some visuals
ans.plot2d()
ans.path2d()
#ans.video3d()  # This saves a video to the folder this notebook is in!

In [ ]:
ans.compare('momentum', lr=lambda t: 1e-4, num_iters=30)

## <span style="color:red">Submission: Challenge</span>
<span style="color:red">Place code in the below cells that demonstrates your results for each challenge. Each cell should end with `get_min_errs()` displaying the achieved error.</span>

### Booth function

In [ ]:
# Replace parameters here
params = dict(
    method='momentum',
    lr=lr_,
    num_iters=15,
    alpha=0.8
)
ans.set_settings(fn_name='booth', x0=np.array([8, 9]), **params)
ans.get_min_errs()

### Beale function

In [ ]:
# Replace parameters here
params = dict(
    method='adagrad',
    lr=lr_,
    num_iters=15,
    alpha=0.8
)
ans.set_settings(fn_name='beale', x0=np.array([8, 9]), **params)
ans.get_min_errs()

### Rosenbrock function

In [ ]:
# Replace parameters here
params = dict(
    method='adagrad',
    lr=lr_,
    num_iters=10,
)
ans.set_settings(fn_name='rosen2d', x0=np.array([2, 9]), **params)
ans.get_min_errs()

### Ackley function

In [ ]:
# Replace parameters here
params = dict(
    method='adam',
    lr=lr_,
    num_iters=15,
)
ans.set_settings(fn_name='ackley2d', x0=np.array([25, 20]), **params)
ans.get_min_errs()

## <span style="color:red">Submission: Project Report</span>

<span style="color:red">Create a report explaining the avenues you explored after the implementation phase of the project, the process you used to select the function values for each combination of functions and initial points, and what you found or learned. You are encouraged to include explanatory images or links to videos generated in the process, showcasing the process you describe or any interesting or unusual phenomena you observe over the course of your investigation!</span>

Please append a PDF print-out of this Jupyter notebook, including any code for extensions you choose to do (see below), to your project report.

## Rubric

- A C level project would successfully implement 4 out of 5 of the optimization algorithms and 3 out of 4 of the benchmark functions (and their derivatives).
- A B level project would successfully implement all the optimization algorithms and benchmark functions (and their derivatives), as well as complete a project report.
- An A level project would do all of the above, plus one or more extensions (we suggest some below, but you can pick anything of sufficient interest and complexity).

Note that exceptional projects that go above and beyond may receive extra credit beyond at our discretion.

### Extensions

Some extensions to this project you could do and include in your project report are:
- Correctly implement more classes of optimization algorithms -- do at least one algorithm for credit, and benchmark its performance. Examples include [proximal gradient descent](http://www.seas.ucla.edu/~vandenbe/236C/lectures/proxgrad.pdf), and [gradient descent with line search](https://optimization.cbe.cornell.edu/index.php?title=Line_search_methods). 
- A "literature review" of optimization algorithms. What this means is to take a research paper or two from the study of optimization algorithms and summarize it (or them) in a way that your peers can understand. A good literature review should contain: an introduction to the proposed method, a formal description of the proposed method, and discussion about why the method is useful or needed. You can use any papers from this field that you please, but here are a few in case you are stuck: ["Nesterov's Accelerated Gradient and Momentum as approximations to Regularised Update Descent"](https://arxiv.org/abs/1607.01981), ["A Universal Catalyst for First-Order Optimization"](https://arxiv.org/abs/1506.02186). You could also discuss one of the papers your TA Tarun wrote: ["A Potential Reduction Inspired Algorithm for Exact Max Flow in Almost O(m^(4/3)) Time"](https://arxiv.org/abs/2009.03260), which has interesting optimization ideas inside an algorithmic framework.
- Describe how the algorithms discussed compare to higher-order algorithms, such as Newton's method and especially interior point methods. For interior point methods, implementation will be hard due to numerical stability issues, so you can do a more theoretical review; but other comparisons should involve implementations of the comparison methods.
- Quantitatively compare how the dimensionality of the problem can affect the algorithms. How do the different algorithms fare in higher dimensions? What benchmarks or visualizations can you use for higher-dimensional optimization?